# <center>Eye Tracking Project</center>

## Table of Contents

1. [Import Requirements](#Import-Requirements)
2. [Context Information](#Context-information)
3. [Paper 1: IPA](#The-Index-of-Pupillary-Activity)
    1. [Version 1](#Version-1) Hardcoded Version 
    2. [Version 2](#Version-2) PyWavelet Version
    
    
## Import Requirements
These are the required packages to make the program work, I tried to limit it to only what was found within the original pseudocode with only a few additional ones where I thought they would be best used.

In [8]:
import math
import numpy as np
import pandas as pd
import pywt

from matplotlib import pyplot as plt

## Context information
I will be trying to implement three pupillometric measures from Duchowski's papers regarding: 
- 1) Index of Pupillary Activity (IPA)
- 2) Low/High Index of Pupillary Activity (LHIPA)
- 3) Reat-Time-Index of Pupillary Activity (RIPA) 

Which are all loosely based on Marshall's Index of Cognitive Activity (ICA) from 2002.

## The Index of Pupillary Activity
The paper Duchowski et al. (2018) is the basis for this and it is the first iteration of their eye-tracked measure of the requency of pupil diameter oscillation. What this paper achieves is creating/replicating the Index of Cognitive Activity (ICA) and in general improving upon it without necessarily being copyrighted. 


## Version 1
This version was created before I was aware that PyWavelet could do 90% of what I needed for the code. This can be found on the Github at this location. But is otherwise not up to date.

## Version 2
This version is the final one that is used within the implementation and is far cleaner code because of it.

### IPA Implementation
There are roughly six formulas that are used within the paper to calculate IPA. Below we will go through each one and how they are implemented, what their variables are, and how they connect to one another. 


### Wavelet Decomposition
All of the five formulas listed below construct the first part of our python implementation from the Duchowski et al. (2018) paper. Each of them will be noted in their explicit part in the end result but are as such not hardcode calculated in this current version of the project.

#### Dyadic Wavelet Function
The first of many functions, this one represents the wavelet function used in the analysis and is representative of the $wavelet$ argument in the function. For this specific rendition of the code, we assume that they are using the "sym16" version as that is what is used within the pseudo-code.

We will do this with the following variables:
- $x(t)$: pupil diameter signal over time
- $\psi(t)$: the mother wavelet function 
    - (Symlet-16)
- $j$: dilation parameter, integers from a set that can represent any number
- $k$: translation parameter, integers from a set that can represent any number
- $2^{j/2}$: Scaling factor w.r.t. time domain
- $\psi(2^j t - k)$: Shifted and scaled wavelet function

$$
\psi_{j,k}(t) = 2^{j/2} \psi(2^j t - k), \quad \text{where } j, k \in \mathbb{Z} \qquad \text{(0)}
$$

#### Integral Transformation of Wavelet Coefficients
This step is far more complicated as it goes through several iterations of it within the Duchowski et al. (2018) paper, but we'll mainly focus on the steps taht we can complete here.

(For the sake of space, repeated variables will not be given descriptions)

##### Decomposition of the Wavelet Analysis w.r.t. coefficients
This formula is implicit to our calculation but is not directly incorporated into our code.

Variable representation:
- $L^2(\mathbb{R})$: Noting that $x(t)$ is a square-integrable function
    - (meaning it won't become infinitely large when squared or integrated)
- $\sum_{j,k = -\infty}^{\infty}$: represents the sum over all possible combinations of j and k that are integers.
- $c_{j,k}$: wavelet coefficients of $\psi_{j,k}(t)$
- $x(t)$: pupil diameter signal
- $\psi_{j,k}(t)$: the mother wavelet function 
    - (Symlet-16)


$$
x \in L^2(\mathbb{R}): x(t) = \sum_{j,k = -\infty}^{\infty} c_{j,k} \psi_{j,k}(t), \quad j,k \in \mathbb{Z} \qquad \text{(1)}
$$

##### Inner Product Calculation of the Wavelet Coefficients
This formula computes the wavelet coefficients by taking the inner product of the input $x(t)$ and tells us how well the function performs at different scales and translations. From this formula onward, these are all explicitly done under the hood of "pywt.wavedec" which is the PyWavelet Decomposition function.

Variable representation:
- $c_{j,k}$: already noted
- $\int_{-\infty}^{\infty}$: Integral of the formula, represents area under the curve
- $x(t)$: already noted
- $\overline{\psi_{j,k}(t)}$: We are taking the complex conjugate of our previous wavelet function. 
    - (this ensures that the result is a real value)
- $dt$: The integration of the variable t

$$
c_{j,k} = \int_{-\infty}^{\infty} x(t) \overline{\psi_{j,k}(t)} dt, \quad x \in L^2(\mathbb{R}), \quad j,k \in \mathbb{Z} \qquad \text{(2)}
$$

##### Substitution Property for Formula (0) and Formula (2)
Variables inlcuded from Formula (0):
- $j$: dilation parameter, integers from a set that can represent any number
- $k$: translation parameter, integers from a set that can represent any number
- $2^{j/2}$: Scaling factor w.r.t. time domain
- $\psi(2^j t - k)$: Shifted and scaled wavelet function

$$
c_{j,k} = 2^{j/2} \int_{-\infty}^{\infty} x(t) \overline{\psi(2^j t - k)} \, dt, \quad x \in L^2(\mathbb{R}), \quad j,k \in \mathbb{Z}
\qquad \text{(3)}
$$

##### The Wavelet Coefficients in their final form
This formula gives us the similarity between the signal and the wavelet function at each scale and position.

Variable representation:
- $\{W_\psi x(t)\}(j, k)$: The wavelet transformation of $x(t)$ using $\psi$
- $\langle x(t), \psi_{j,k}(t) \rangle$: The inner product reprsented in another way from formula (3)

$$
= \{W_\psi x(t)\}(j, k) = \langle x(t), \psi_{j,k}(t) \rangle
\qquad \text{(4)}
$$

In [79]:
class PyPil:
    """
    Purpose: 
    This class is used to calculate three types of formulas relating to pupil measurement. 
    Each serves as an extension/modification of the previous one and as such is listed in the corresponding 
        order within the class. 
    1) Index of Pupillary Activity (IPA) from Duchowski et al. (2018)
    2) Low/High Index of Pupillary Activity (LHIPA) from Duchowski et al. (2020)
    3) Reat-Time-Index of Pupillary Activity (RIPA) from Jayawardena et al. (2022)
    
    ...
    
    Attributes:
    pupil_data : list
        The pupil diameter measurements in pixels
    wavelet : str
        The wavelet chosen for the decomposition (default: sym16)
    periodization : str
        The periodization chosen for the decomposition (default: per)
    level : str
        The level of granularity done within the decomposition (default: 2)
        
    ...
    
    Methods:
    wavelet_decomposition()
        This returns the coefficients of the wavelet decomposition.
    
    scaling_function_analysis()
        
    
    
    """
    def __init__(self, pupil_data, wavelet="sym16", periodization="per", level = 2):
        try:
            import numpy as np
            import math
            import pywt
        except ImportError as e:
            raise ImportError("A required library is not imported, verify that numpy, math, and pywt are imported.")
        
        self.pupil_data_diam = list(pupil_data['diameter'])
        self.pupil_data_time = list(pupil_data['world_index'])
        self.wavelet = wavelet
        self.periodization = periodization
        self.level = level
        
    def wavelet_decomposition(self):
        try:
            cA2, cD2, cD1 = pywt.wavedec(self.pupil_data_diam, self.wavelet, mode=self.periodization, level=self.level)
#             print("Wavelet Decomposition:")
#             print("\n", "cA2:",  cA2)
#             print("\n", "cD2:", cD2)
#             print("\n", "cD1:", cD1)
            return cA2, cD2, cD1 
        except ValueError:
            return None

    # get signal duration (in seconds) 
    # tt = d[-1].timestamp() - d[0].timestamp()
    def calculate_timestamps(self):
        tt = self.pupil_data_time[-1] - self.pupil_data_time[0]
#         print("\nTimestamp Calculation:")
#         print("tt:", tt)
        return float(tt)
        
    def normalize_coefficients(self, cA2, cD2, cD1, level):
        # normalize by 1/2 j , j = 2 for 2- level DWT 
        cA2[:] = [x / math.sqrt(2.0 * level) for x in cA2] 
        cD1[:] = [x / math.sqrt(1.0 * level) for x in cD1] 
        cD2[:] = [x / math.sqrt(2.0 * level) for x in cD2]
        return cA2[:], cD2[:], cD1[:]

    def compute_modmax(self, cD2):
        sig_modu = [math.fabs(x) for x in cD2]
        
        detect = [0.0] * len(cD2)
        for i in range(len(cD2)):
            ll = sig_modu[i - 1] if i >= 1 else sig_modu[i]
            oo = sig_modu[i]
            rr = sig_modu[i + 1] if i < len(cD2)-2 else sig_modu[i]
            if (ll <= oo and oo >= rr) and (ll < oo or oo > rr):
                detect[i] = math.sqrt(self.pupil_data_diam[i]**2)
            else:
                detect[i] = 0.0
#         print("\nModmax Calculation:")
#         print(detect)
        return detect

    def compute_threshold(self, detect, mode="hard"):
        thresh = np.std(detect) * math.sqrt(2.0*np.log2(len(detect)))
        cD2t = pywt.threshold(detect,thresh,mode)
#         print("\nCalculate Threshold:")
#         print(cD2t)
        return cD2t
    
    def calculate_ipa(self, cD2t):
        ctr = 0
        for i in range(len(cD2t)):
            if math.fabs(cD2t[i]) > 0: 
                ctr += 1
        ipa = float(ctr)/ self.calculate_timestamps()
#         print("\nIPA:")
#         print(ipa)
        return ipa
    
    def compute_ipa(self):
        cA2, cD2, cD1  = self.wavelet_decomposition()
        cA2[:], cD2[:], cD1[:], = self.normalize_coefficients(cA2, cD2, cD1, self.level)
        detect = self.compute_modmax(cD2[:])
        cD2t = self.compute_threshold(detect)
        ipa = self.calculate_ipa(cD2t)
        return ipa   

In [80]:
df1 = pd.read_csv('left_eye_pupil_measures.csv')
tester = PyPil(df1)
result = tester.compute_ipa()
print(result)

0.0


In [81]:
if np.all(result == 0):
    print("All elements are zero")
else:
    print("all elements are not zero")

All elements are zero
